In [47]:
import pandas as pd
import os
import numpy as np
import matplotlib.ticker as mtick
import arcpy

# Import the required ArcGIS API for Python modules
import arcgis
gis = arcgis.GIS()

from IPython.display import display, Markdown

In [48]:
#get root folder
dir_working = os.getcwd()
print(dir_working)

#define folders
dir_data     = os.path.join(dir_working, "data"        )
dir_process  = os.path.join(dir_working, "intermediate")
dir_results  = os.path.join(dir_working, "results"     )
dir_inputs   = os.path.join(dir_working, "inputs"      )

ClassParameters    = os.path.join(dir_inputs, r"class_parameters_scenario1.csv")
CountiesTable      = os.path.join(dir_inputs ,r"counties.csv")

ProcessGDB = "process.gdb"

#name of new data features
AnalysisAreas      = os.path.join(dir_process, "analysisareas.shp"        )
AnalysisAreas_IDs  = os.path.join(dir_process, "analysisareas_ids.shp"    )
ParcelsAA          = os.path.join(dir_process, "parcelsaa.shp"            )

AnalysisAreasTable = os.path.join(dir_results, "areas.csv"                )



def deleteIfExists(obj):
    if arcpy.Exists(obj): arcpy.Delete_management(obj)


e:\GitHub\Centers-Capacity-Tool-byCity


In [49]:
AnalysisAreas

'e:\\GitHub\\Centers-Capacity-Tool-byCity\\intermediate\\analysisareas.shp'

In [50]:
sdf_AA = pd.DataFrame.spatial.from_featureclass(AnalysisAreas)
#sdf_AA

sdf_AAid = pd.DataFrame.spatial.from_featureclass(AnalysisAreas_IDs)
sdf_AAid

,FID,Shape_Leng,Shape_Area,area_id,class_id,SHAPE
0,0,0.002517,1.025227e-07,3,8,"{""rings"": [[[-111.93828708299998, 40.500645842..."
1,1,0.014422,1.756577e-06,4,8,"{""rings"": [[[-111.94214447999997, 40.460508711..."
2,2,0.081911,3.457529e-04,5,8,"{""rings"": [[[-111.90309291399996, 40.486177586..."
3,3,0.032693,6.718693e-07,43,8,"{""rings"": [[[-112.04360043899999, 40.516117516..."
4,4,0.030717,5.078766e-05,44,8,"{""rings"": [[[-111.94214447999997, 40.460508711..."
...,...,...,...,...,...,...
289,289,0.160173,2.067631e-04,147,9,"{""rings"": [[[-111.90748598099998, 40.768529726..."
290,290,0.031583,7.897752e-05,147,1,"{""rings"": [[[-111.90269131299999, 40.762673174..."
291,291,0.030149,4.829866e-05,147,1,"{""rings"": [[[-111.89878362799999, 40.772698415..."
292,292,0.085443,2.256532e-04,147,5,"{""rings"": [[[-111.88250224899997, 40.763937603..."


In [51]:
#create map centered on Salt Lake
map_areas = gis.map('Salt Lake')

#expression to classify enrollment
arcade_expression_aa = ("return 'classAA';")

#symbology for enrollment classes
uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[128,128,128,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]

#classify land use
arcade_expression_aa = ("var v = $feature.Analysis_1;"
                        "if      (v=='NONTOD|Metropolitan Center') { return 'mc'    ; }"
                        "else if (v=='CRT|Metropolitan Center'   ) { return 'mc_tod'; }"
                        "else if (v=='LRT|Metropolitan Center'   ) { return 'mc_tod'; }"
                        "else if (v=='NONTOD|Urban Center'       ) { return 'uc'    ; }"
                        "else if (v=='CRT|Urban Center'          ) { return 'uc_tod'; }"
                        "else if (v=='LRT|Urban Center'          ) { return 'uc_tod'; }"
                        "else if (v=='NONTOD|City Center'        ) { return 'cc'    ; }"
                        "else if (v=='CRT|City Center'           ) { return 'cc_tod'; }"
                        "else if (v=='LRT|City Center'           ) { return 'cc_tod'; }"
                        "else if (v=='CRT|NA'                    ) { return 'nc_tod'; }"
                        "else if (v=='LRT|NA'                    ) { return 'nc_tod'; }"
                        "else                                      { return 'other' ; }")

#symbology for enrollment classes
uv_aa = [  
          {"value":"mc"    , "label":"Metropolitan Center"         , "symbol":{"type":"esriSFS","color":[128,  0,128,100], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"mc_tod", "label":"Metropolitan Center with TOD", "symbol":{"type":"esriSFS","color":[128,  0,128,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"uc"    , "label":"Urban Center"                , "symbol":{"type":"esriSFS","color":[  0,  0,128,100], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"uc_tod", "label":"Urban Center with TOD"       , "symbol":{"type":"esriSFS","color":[  0,  0,128,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"cc"    , "label":"City Center"                 , "symbol":{"type":"esriSFS","color":[255,  0,  0,100], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"cc_tod", "label":"City Center with TOD"        , "symbol":{"type":"esriSFS","color":[255,  0,  0,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"nc_tod", "label":"TOD Only"                    , "symbol":{"type":"esriSFS","color":[  0,255,  0,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"other" , "label":"Other"                       , "symbol":{"type":"esriSFS","color":[128,128,128,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}
        ]

#define sdf layer
sdf_AA.spatial.plot(map_widget = map_areas,
                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                 unique_values=uv_aa,
                                 arcade_expression=arcade_expression_aa,
                                 default_symbol="" #don't include an 'other' category
                                 )

#define map characteristics
map_areas.layout.height='500px'
map_areas.legend=True

#map title
display(Markdown('<h2><center>Analysis Areas</center></h2>'))

#display map
map_areas

<h2><center>Analysis Areas</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

# Parameters

In [52]:
df_ClassParam = pd.read_csv(ClassParameters)
display(df_ClassParam)

,ClassID,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,ClassOrder
0,1,CRT|Metropolitan Center,0.40,0.60,350,8,1000,500,0.05,650000,400000,AND,60,120,30,120,1
1,5,LRT|Metropolitan Center,0.40,0.60,350,8,1000,500,0.05,650000,400000,AND,60,120,30,120,2
2,9,NONTOD|Metropolitan Center,0.45,0.55,170,4,1000,500,0.05,650000,400000,AND,60,120,30,120,3
3,3,CRT|Urban Center,0.50,0.50,130,3,1000,500,0.10,500000,300000,AND,70,300,30,300,4
4,7,LRT|Urban Center,0.50,0.50,130,3,1000,500,0.10,500000,300000,AND,70,300,30,300,5
5,10,NONTOD|Urban Center,0.55,0.45,70,2,1200,500,0.10,500000,300000,AND,70,300,30,300,6
6,0,CRT|City Center,0.60,0.40,70,2,1200,600,0.10,500000,300000,AND,70,300,30,300,7
7,4,LRT|City Center,0.60,0.40,70,2,1200,600,0.10,500000,300000,AND,70,300,30,300,8
8,8,NONTOD|City Center,0.65,0.35,35,1,1200,600,0.10,500000,300000,AND,80,300,40,300,9
9,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10


In [53]:
df_ClassParam.dtypes

ClassID                   int64
ClassDescription         object
SFSplitRes              float64
SFSplitCom              float64
CapacityRes_DUA           int64
CapacityCom_FAR           int64
SFperHH                   int64
SFperEmp                  int64
PercentOpenSpace        float64
RedevValuePerAcreRes      int64
RedevValuePerAcreCom      int64
RedevAndOr               object
RedevBldgAgeRes_Low       int64
RedevBldgAgeRes_High      int64
RedevBldgAgeCom_Low       int64
RedevBldgAgeCom_High      int64
ClassOrder                int64
dtype: object

In [54]:
df_AADevCat = pd.read_csv(os.path.join(dir_results, r'AnalysisAreas_byDevCategory.csv'))
df_AADevCat

,area_id,class_id,county_id,dev_category,parcel_id,job_spaces,resunits,acres,sf_res,sf_com,acres_res,acres_com
0,0,2,4.0,Developable,46.0,0.000000,0.000000,30.777796,0.000000,0.000000,0.000000,0.000000
1,0,2,4.0,Remain,21.0,0.000000,6.391683,6.426645,25060.942589,0.000000,6.426645,0.000000
2,0,2,4.0,Undevelopable,22.0,0.000000,0.061490,6.743387,180.410686,0.000000,0.000000,0.000000
3,1,0,4.0,Developable,98.0,0.000000,0.000000,21.879749,0.000000,0.000000,0.000000,0.000000
4,1,0,4.0,Remain,33.0,492.716357,3.413275,22.210932,7539.391832,184465.743750,0.894349,21.316583
...,...,...,...,...,...,...,...,...,...,...,...,...
918,222,8,1.0,Undevelopable,11.0,0.000000,0.000000,6.111900,0.000000,0.000000,0.000000,0.000000
919,223,2,1.0,Developable,58.0,0.000000,0.000000,33.147583,0.000000,0.000000,0.000000,0.000000
920,223,2,1.0,Redevelopable,9.0,26.051029,3.999483,5.457562,6689.134478,24977.482447,1.389284,4.068278
921,223,2,1.0,Remain,334.0,388.671084,296.733379,72.605319,371485.457837,206880.106991,61.060865,11.544454


# Calculate New Jobs/HH

In [55]:
#join Analysis Areas with class parameters
df_AADevCatwParam = pd.DataFrame.merge(df_AADevCat,df_ClassParam,left_on='class_id',right_on='ClassID',how='left')
df_AADevCatwParam

,area_id,class_id,county_id,dev_category,parcel_id,job_spaces,resunits,acres,sf_res,sf_com,acres_res,acres_com,ClassID,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,ClassOrder
0,0,2,4.0,Developable,46.0,0.000000,0.000000,30.777796,0.000000,0.000000,0.000000,0.000000,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10
1,0,2,4.0,Remain,21.0,0.000000,6.391683,6.426645,25060.942589,0.000000,6.426645,0.000000,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10
2,0,2,4.0,Undevelopable,22.0,0.000000,0.061490,6.743387,180.410686,0.000000,0.000000,0.000000,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10
3,1,0,4.0,Developable,98.0,0.000000,0.000000,21.879749,0.000000,0.000000,0.000000,0.000000,0,CRT|City Center,0.60,0.40,70,2,1200,600,0.10,500000,300000,AND,70,300,30,300,7
4,1,0,4.0,Remain,33.0,492.716357,3.413275,22.210932,7539.391832,184465.743750,0.894349,21.316583,0,CRT|City Center,0.60,0.40,70,2,1200,600,0.10,500000,300000,AND,70,300,30,300,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,222,8,1.0,Undevelopable,11.0,0.000000,0.000000,6.111900,0.000000,0.000000,0.000000,0.000000,8,NONTOD|City Center,0.65,0.35,35,1,1200,600,0.10,500000,300000,AND,80,300,40,300,9
919,223,2,1.0,Developable,58.0,0.000000,0.000000,33.147583,0.000000,0.000000,0.000000,0.000000,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10
920,223,2,1.0,Redevelopable,9.0,26.051029,3.999483,5.457562,6689.134478,24977.482447,1.389284,4.068278,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10
921,223,2,1.0,Remain,334.0,388.671084,296.733379,72.605319,371485.457837,206880.106991,61.060865,11.544454,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10


In [56]:
#remove percent open space from available acreage
df_AADevCatwParam['AcresOpenSpaceRemoved'] = df_AADevCatwParam['acres'] * (1-df_AADevCatwParam['PercentOpenSpace'])
pd.set_option('display.max_columns', None)
display(df_AADevCatwParam)
display(df_AADevCatwParam.columns)

,area_id,class_id,county_id,dev_category,parcel_id,job_spaces,resunits,acres,sf_res,sf_com,acres_res,acres_com,ClassID,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,ClassOrder,AcresOpenSpaceRemoved
0,0,2,4.0,Developable,46.0,0.000000,0.000000,30.777796,0.000000,0.000000,0.000000,0.000000,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10,26.161127
1,0,2,4.0,Remain,21.0,0.000000,6.391683,6.426645,25060.942589,0.000000,6.426645,0.000000,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10,5.462648
2,0,2,4.0,Undevelopable,22.0,0.000000,0.061490,6.743387,180.410686,0.000000,0.000000,0.000000,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10,5.731879
3,1,0,4.0,Developable,98.0,0.000000,0.000000,21.879749,0.000000,0.000000,0.000000,0.000000,0,CRT|City Center,0.60,0.40,70,2,1200,600,0.10,500000,300000,AND,70,300,30,300,7,19.691774
4,1,0,4.0,Remain,33.0,492.716357,3.413275,22.210932,7539.391832,184465.743750,0.894349,21.316583,0,CRT|City Center,0.60,0.40,70,2,1200,600,0.10,500000,300000,AND,70,300,30,300,7,19.989839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,222,8,1.0,Undevelopable,11.0,0.000000,0.000000,6.111900,0.000000,0.000000,0.000000,0.000000,8,NONTOD|City Center,0.65,0.35,35,1,1200,600,0.10,500000,300000,AND,80,300,40,300,9,5.500710
919,223,2,1.0,Developable,58.0,0.000000,0.000000,33.147583,0.000000,0.000000,0.000000,0.000000,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10,28.175446
920,223,2,1.0,Redevelopable,9.0,26.051029,3.999483,5.457562,6689.134478,24977.482447,1.389284,4.068278,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10,4.638928
921,223,2,1.0,Remain,334.0,388.671084,296.733379,72.605319,371485.457837,206880.106991,61.060865,11.544454,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10,61.714521


Index(['area_id', 'class_id', 'county_id', 'dev_category', 'parcel_id',
       'job_spaces', 'resunits', 'acres', 'sf_res', 'sf_com', 'acres_res',
       'acres_com', 'ClassID', 'ClassDescription', 'SFSplitRes', 'SFSplitCom',
       'CapacityRes_DUA', 'CapacityCom_FAR', 'SFperHH', 'SFperEmp',
       'PercentOpenSpace', 'RedevValuePerAcreRes', 'RedevValuePerAcreCom',
       'RedevAndOr', 'RedevBldgAgeRes_Low', 'RedevBldgAgeRes_High',
       'RedevBldgAgeCom_Low', 'RedevBldgAgeCom_High', 'ClassOrder',
       'AcresOpenSpaceRemoved'],
      dtype='object')

In [57]:
#calculate Com and Res Acreage

#initialize - acres_remain and acres_developed don't have res/com breakdown
df_AADevCatwParam['acres_undevelopable'] = 0.0
df_AADevCatwParam['acres_remain'       ] = 0.0
df_AADevCatwParam['acres_redevelopable'] = 0.0
df_AADevCatwParam['acres_developable'  ] = 0.0

#set acres
df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Undevelopable'), 'acres_undevelopable'] = df_AADevCatwParam['acres']
df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Remain'       ), 'acres_remain'       ] = df_AADevCatwParam['acres']
df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Redevelopable'), 'acres_redevelopable'] = df_AADevCatwParam['acres']
df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Developable')  , 'acres_developable'  ] = df_AADevCatwParam['acres']


#calculate Com and Res SF


#initialize
df_AADevCatwParam['SFcom_undevelopable'   ] = 0.0
df_AADevCatwParam['SFres_undevelopable'   ] = 0.0

df_AADevCatwParam['SFcom_remain'          ] = 0.0
df_AADevCatwParam['SFres_remain'          ] = 0.0

df_AADevCatwParam['SFcom_redeveloped_orig'] = 0.0
df_AADevCatwParam['SFres_redeveloped_orig'] = 0.0

df_AADevCatwParam['SFcom_redeveloped_new' ] = 0.0
df_AADevCatwParam['SFres_redeveloped_new' ] = 0.0

df_AADevCatwParam['SFcom_developed'       ] = 0.0
df_AADevCatwParam['SFres_developed'       ] = 0.0


#set undevelopable sf
df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Undevelopable'), 'SFcom_undevelopable'     ] = df_AADevCatwParam['sf_com']
df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Undevelopable'), 'SFres_undevelopable'     ] = df_AADevCatwParam['sf_res']

#set remaining sf
df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Remain'       ), 'SFcom_remain'            ] = df_AADevCatwParam['sf_com']
df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Remain'       ), 'SFres_remain'            ] = df_AADevCatwParam['sf_res']


# SFCom = AcresNoOpenSpace / ((SPLITres/SPLITcom)/(HHperAcre*SFperHH) + (1/(FAR*43560)))
# SFRes = SFCom * SPLITres/SPLITcom
df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Redevelopable'), 'SFcom_redeveloped_orig'  ] = df_AADevCatwParam['sf_com']
df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Redevelopable'), 'SFres_redeveloped_orig'  ] = df_AADevCatwParam['sf_res']

df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Redevelopable'), 'SFcom_redeveloped_new'   ] = (df_AADevCatwParam['AcresOpenSpaceRemoved'] / ((df_AADevCatwParam['SFSplitRes']/df_AADevCatwParam['SFSplitCom'])/(df_AADevCatwParam['CapacityRes_DUA']*df_AADevCatwParam['SFperHH']) + (1/(df_AADevCatwParam['CapacityCom_FAR']*43560)))).astype('int64')
df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Redevelopable'), 'SFres_redeveloped_new'   ] = (df_AADevCatwParam['SFcom_redeveloped_new'] * df_AADevCatwParam['SFSplitRes'] / df_AADevCatwParam['SFSplitCom']).astype('int64')

df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Developable'  ), 'SFcom_developed'         ] = (df_AADevCatwParam['AcresOpenSpaceRemoved'] / ((df_AADevCatwParam['SFSplitRes']/df_AADevCatwParam['SFSplitCom'])/(df_AADevCatwParam['CapacityRes_DUA']*df_AADevCatwParam['SFperHH']) + (1/(df_AADevCatwParam['CapacityCom_FAR']*43560)))).astype('int64')
df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Developable'  ), 'SFres_developed'         ] = (df_AADevCatwParam['SFcom_developed']       * df_AADevCatwParam['SFSplitRes'] / df_AADevCatwParam['SFSplitCom']).astype('int64')


df_AADevCatwParam

,area_id,class_id,county_id,dev_category,parcel_id,job_spaces,resunits,acres,sf_res,sf_com,acres_res,acres_com,ClassID,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,ClassOrder,AcresOpenSpaceRemoved,acres_undevelopable,acres_remain,acres_redevelopable,acres_developable,SFcom_undevelopable,SFres_undevelopable,SFcom_remain,SFres_remain,SFcom_redeveloped_orig,SFres_redeveloped_orig,SFcom_redeveloped_new,SFres_redeveloped_new,SFcom_developed,SFres_developed
0,0,2,4.0,Developable,46.0,0.000000,0.000000,30.777796,0.000000,0.000000,0.000000,0.000000,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10,26.161127,0.000000,0.000000,0.000000,30.777796,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,333210.0,777489.0
1,0,2,4.0,Remain,21.0,0.000000,6.391683,6.426645,25060.942589,0.000000,6.426645,0.000000,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10,5.462648,0.000000,6.426645,0.000000,0.000000,0.000000,0.000000,0.000000,25060.942589,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0,2,4.0,Undevelopable,22.0,0.000000,0.061490,6.743387,180.410686,0.000000,0.000000,0.000000,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10,5.731879,6.743387,0.000000,0.000000,0.000000,0.000000,180.410686,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,1,0,4.0,Developable,98.0,0.000000,0.000000,21.879749,0.000000,0.000000,0.000000,0.000000,0,CRT|City Center,0.60,0.40,70,2,1200,600,0.10,500000,300000,AND,70,300,30,300,7,19.691774,0.000000,0.000000,0.000000,21.879749,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,671259.0,1006888.0
4,1,0,4.0,Remain,33.0,492.716357,3.413275,22.210932,7539.391832,184465.743750,0.894349,21.316583,0,CRT|City Center,0.60,0.40,70,2,1200,600,0.10,500000,300000,AND,70,300,30,300,7,19.989839,0.000000,22.210932,0.000000,0.000000,0.000000,0.000000,184465.743750,7539.391832,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,222,8,1.0,Undevelopable,11.0,0.000000,0.000000,6.111900,0.000000,0.000000,0.000000,0.000000,8,NONTOD|City Center,0.65,0.35,35,1,1200,600,0.10,500000,300000,AND,80,300,40,300,9,5.500710,6.111900,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
919,223,2,1.0,Developable,58.0,0.000000,0.000000,33.147583,0.000000,0.000000,0.000000,0.000000,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10,28.175446,0.000000,0.000000,0.000000,33.147583,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,358866.0,837354.0
920,223,2,1.0,Redevelopable,9.0,26.051029,3.999483,5.457562,6689.134478,24977.482447,1.389284,4.068278,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10,4.638928,0.000000,0.000000,5.457562,0.000000,0.000000,0.000000,0.000000,0.000000,24977.482447,6689.134478,59085.0,137865.0,0.0,0.0
921,223,2,1.0,Remain,334.0,388.671084,296.733379,72.605319,371485.457837,206880.106991,61.060865,11.544454,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10,61.714521,0.000000,72.605319,0.000000,0.000000,0.000000,0.000000,206880.106991,371485.457837,0.000000,0.000000,0.0,0.0,0.0,0.0


In [58]:
#calculate jobs and HH

#initialize
df_AADevCatwParam['job_spaces_undevelopable'   ] = 0.0
df_AADevCatwParam['job_spaces_remain'          ] = 0.0
df_AADevCatwParam['job_spaces_redeveloped_orig'] = 0.0
df_AADevCatwParam['job_spaces_redeveloped_new' ] = 0.0
df_AADevCatwParam['job_spaces_developed'       ] = 0.0

df_AADevCatwParam['resunits_undevelopable'     ] = 0.0
df_AADevCatwParam['resunits_remain'            ] = 0.0
df_AADevCatwParam['resunits_redeveloped_orig'  ] = 0.0
df_AADevCatwParam['resunits_redeveloped_new'   ] = 0.0
df_AADevCatwParam['resunits_developed'         ] = 0.0

#set undevelopable sf
df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Undevelopable'), 'job_spaces_undevelopable'   ] =  df_AADevCatwParam['job_spaces'           ]
df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Remain'       ), 'job_spaces_remain'          ] =  df_AADevCatwParam['job_spaces'           ]
df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Redevelopable'), 'job_spaces_redeveloped_orig'] =  df_AADevCatwParam['job_spaces'           ]
df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Redevelopable'), 'job_spaces_redeveloped_new' ] = (df_AADevCatwParam['SFcom_redeveloped_new'] / df_AADevCatwParam['SFperEmp']).astype('int64')
df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Developable'  ), 'job_spaces_developed'       ] = (df_AADevCatwParam['SFcom_developed'      ] / df_AADevCatwParam['SFperEmp']).astype('int64')

df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Undevelopable'), 'resunits_undevelopable'     ] =  df_AADevCatwParam['resunits'             ]
df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Remain'       ), 'resunits_remain'            ] =  df_AADevCatwParam['resunits'             ]
df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Redevelopable'), 'resunits_redeveloped_orig'  ] =  df_AADevCatwParam['resunits'             ]
df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Redevelopable'), 'resunits_redeveloped_new'   ] = (df_AADevCatwParam['SFres_redeveloped_new'] / df_AADevCatwParam['SFperHH' ]).astype('int64')
df_AADevCatwParam.loc[(df_AADevCatwParam['dev_category'] == 'Developable'  ), 'resunits_developed'         ] = (df_AADevCatwParam['SFres_developed'      ] / df_AADevCatwParam['SFperHH' ]).astype('int64')


df_AADevCatwParam

,area_id,class_id,county_id,dev_category,parcel_id,job_spaces,resunits,acres,sf_res,sf_com,acres_res,acres_com,ClassID,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,ClassOrder,AcresOpenSpaceRemoved,acres_undevelopable,acres_remain,acres_redevelopable,acres_developable,SFcom_undevelopable,SFres_undevelopable,SFcom_remain,SFres_remain,SFcom_redeveloped_orig,SFres_redeveloped_orig,SFcom_redeveloped_new,SFres_redeveloped_new,SFcom_developed,SFres_developed,job_spaces_undevelopable,job_spaces_remain,job_spaces_redeveloped_orig,job_spaces_redeveloped_new,job_spaces_developed,resunits_undevelopable,resunits_remain,resunits_redeveloped_orig,resunits_redeveloped_new,resunits_developed
0,0,2,4.0,Developable,46.0,0.000000,0.000000,30.777796,0.000000,0.000000,0.000000,0.000000,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10,26.161127,0.000000,0.000000,0.000000,30.777796,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,333210.0,777489.0,0.000000,0.000000,0.000000,0.0,555.0,0.00000,0.000000,0.000000,0.0,647.0
1,0,2,4.0,Remain,21.0,0.000000,6.391683,6.426645,25060.942589,0.000000,6.426645,0.000000,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10,5.462648,0.000000,6.426645,0.000000,0.000000,0.000000,0.000000,0.000000,25060.942589,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00000,6.391683,0.000000,0.0,0.0
2,0,2,4.0,Undevelopable,22.0,0.000000,0.061490,6.743387,180.410686,0.000000,0.000000,0.000000,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10,5.731879,6.743387,0.000000,0.000000,0.000000,0.000000,180.410686,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.06149,0.000000,0.000000,0.0,0.0
3,1,0,4.0,Developable,98.0,0.000000,0.000000,21.879749,0.000000,0.000000,0.000000,0.000000,0,CRT|City Center,0.60,0.40,70,2,1200,600,0.10,500000,300000,AND,70,300,30,300,7,19.691774,0.000000,0.000000,0.000000,21.879749,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,671259.0,1006888.0,0.000000,0.000000,0.000000,0.0,1118.0,0.00000,0.000000,0.000000,0.0,839.0
4,1,0,4.0,Remain,33.0,492.716357,3.413275,22.210932,7539.391832,184465.743750,0.894349,21.316583,0,CRT|City Center,0.60,0.40,70,2,1200,600,0.10,500000,300000,AND,70,300,30,300,7,19.989839,0.000000,22.210932,0.000000,0.000000,0.000000,0.000000,184465.743750,7539.391832,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,492.716357,0.000000,0.0,0.0,0.00000,3.413275,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,222,8,1.0,Undevelopable,11.0,0.000000,0.000000,6.111900,0.000000,0.000000,0.000000,0.000000,8,NONTOD|City Center,0.65,0.35,35,1,1200,600,0.10,500000,300000,AND,80,300,40,300,9,5.500710,6.111900,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0
919,223,2,1.0,Developable,58.0,0.000000,0.000000,33.147583,0.000000,0.000000,0.000000,0.000000,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10,28.175446,0.000000,0.000000,0.000000,33.147583,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,358866.0,837354.0,0.000000,0.000000,0.000000,0.0,598.0,0.00000,0.000000,0.000000,0.0,697.0
920,223,2,1.0,Redevelopable,9.0,26.051029,3.999483,5.457562,6689.134478,24977.482447,1.389284,4.068278,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10,4.638928,0.000000,0.000000,5.457562,0.000000,0.000000,0.000000,0.000000,0.000000,24977.482447,6689.134478,59085.0,137865.0,0.0,0.0,0.000000,0.000000,26.051029,98.0,0.0,0.00000,0.

In [59]:
#calculate totals

df_AASummary = df_AADevCatwParam.groupby(['county_id','area_id','class_id','ClassDescription','ClassOrder'], as_index=False).agg({"dev_category":[np.size], "parcel_id":[np.sum], "acres":[np.sum], "acres_undevelopable":[np.sum], "acres_remain":[np.sum], "acres_redevelopable":[np.sum], "acres_developable":[np.sum], "SFcom_undevelopable":[np.sum], "SFres_undevelopable":[np.sum], "SFcom_remain":[np.sum], "SFres_remain":[np.sum], "SFcom_redeveloped_orig":[np.sum], "SFres_redeveloped_orig":[np.sum], "SFcom_redeveloped_new":[np.sum], "SFres_redeveloped_new":[np.sum], "SFcom_developed":[np.sum], "SFres_developed":[np.sum], "job_spaces_undevelopable":[np.sum], "job_spaces_remain":[np.sum], "job_spaces_redeveloped_orig":[np.sum], "job_spaces_redeveloped_new":[np.sum], "job_spaces_developed":[np.sum], "resunits_undevelopable":[np.sum], "resunits_remain":[np.sum], "resunits_redeveloped_orig":[np.sum], "resunits_redeveloped_new":[np.sum], "resunits_developed":[np.sum]})
df_AASummary.columns = df_AASummary.columns.droplevel(1)

df_AASummaryTotals = df_AASummary.copy()

df_AASummaryTotals['job_spaces_orig'  ] = (df_AASummaryTotals['job_spaces_undevelopable'] + df_AASummaryTotals['job_spaces_remain'] + df_AASummaryTotals['job_spaces_redeveloped_orig']).astype('int64')
df_AASummaryTotals['job_spaces_new'   ] = (df_AASummaryTotals['job_spaces_undevelopable'] + df_AASummaryTotals['job_spaces_remain'] + df_AASummaryTotals['job_spaces_redeveloped_new' ] + df_AASummaryTotals['job_spaces_developed']).astype('int64')
df_AASummaryTotals['job_spaces_change'] = (df_AASummaryTotals['job_spaces_new'          ] - df_AASummaryTotals['job_spaces_orig'  ]).round(0)

df_AASummaryTotals['resunits_orig'    ] = (df_AASummaryTotals['resunits_undevelopable'  ] + df_AASummaryTotals['resunits_remain'  ] + df_AASummaryTotals['resunits_redeveloped_orig'  ]).astype('int64')
df_AASummaryTotals['resunits_new'     ] = (df_AASummaryTotals['resunits_undevelopable'  ] + df_AASummaryTotals['resunits_remain'  ] + df_AASummaryTotals['resunits_redeveloped_new'   ] + df_AASummaryTotals['resunits_developed'  ]).astype('int64')
df_AASummaryTotals['resunits_change'  ] = (df_AASummaryTotals['resunits_new'            ] - df_AASummaryTotals['resunits_orig'    ]).round(0)

df_AASummaryTotals = df_AASummaryTotals[['county_id','area_id','class_id','ClassDescription','ClassOrder','parcel_id','acres','acres_redevelopable','acres_developable','job_spaces_orig','job_spaces_new','job_spaces_change','resunits_orig','resunits_new','resunits_change','job_spaces_undevelopable','job_spaces_remain','job_spaces_redeveloped_orig','job_spaces_redeveloped_new','job_spaces_developed','resunits_undevelopable','resunits_remain','resunits_redeveloped_orig','resunits_redeveloped_new','resunits_developed']]

display(df_AASummaryTotals.sum())

df_AASummaryTotals['density_hhemp_orig'  ] = ((df_AASummaryTotals['job_spaces_orig'  ] + df_AASummaryTotals['resunits_orig'     ]) / df_AASummaryTotals['acres']).round(1)
df_AASummaryTotals['density_hhemp_new'   ] = ((df_AASummaryTotals['job_spaces_new'   ] + df_AASummaryTotals['resunits_new'      ]) / df_AASummaryTotals['acres']).round(1)
df_AASummaryTotals['density_hhemp_change'] =  (df_AASummaryTotals['density_hhemp_new'] - df_AASummaryTotals['density_hhemp_orig']).round(1)


county_id                                                                  566.0
area_id                                                                    29716
class_id                                                                    1752
ClassDescription               NONTOD|City CenterNONTOD|City CenterNONTOD|Cit...
ClassOrder                                                                  2236
parcel_id                                                               126288.0
acres                                                               34965.770447
acres_redevelopable                                                  2137.593124
acres_developable                                                   10304.425366
job_spaces_orig                                                           481641
job_spaces_new                                                            999939
job_spaces_change                                                         518298
resunits_orig               

In [60]:
#create map centered on Salt Lake
map_parcels_areas_den = gis.map('Salt Lake')

sdf_AADevCatwParam = pd.DataFrame.merge(sdf_AAid, df_AASummaryTotals, on=['area_id','class_id'], how='left')

#expression to classify employment growth
arcade_expression_denchange = ("var v = $feature.density_hhemp_change;"
                                     "if      (v<0    ) { return 'class0'; }"
                                     "if      (v<20   ) { return 'class1'; }"
                                     "else if (v<50   ) { return 'class2'; }"
                                     "else if (v<100  ) { return 'class3'; }"
                                     "else              { return 'class4'; }")

#symbology for enrollment classes
uv_denchange = [
                #{"value":"class0", "label":"Density Decline", "symbol":{"type":"esriSFS","color":[  0,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                {"value":"class1", "label":"0 to 20"      , "symbol":{"type":"esriSFS","color":[130,165,217,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                {"value":"class2", "label":"20 to 50"     , "symbol":{"type":"esriSFS","color":[140,125,164,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                {"value":"class3", "label":"50 to 100"    , "symbol":{"type":"esriSFS","color":[149, 85,111,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                {"value":"class4", "label":"More than 100", "symbol":{"type":"esriSFS","color":[168,  4,  4,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}
               ]


#define sdf layer
sdf_AADevCatwParam.spatial.plot(map_widget = map_parcels_areas_den,
                                renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                unique_values=uv_denchange,
                                arcade_expression=arcade_expression_denchange,
                                default_symbol="" #don't include an 'other' category
                                )

#define map characteristics
map_parcels_areas_den.layout.height='500px'
map_parcels_areas_den.legend=True

#map title
display(Markdown('<h2><center>Density Change by Analysis Area</center></h2>'))

#display map
map_parcels_areas_den

<h2><center>Density Change by Analysis Area</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

# Area Summary

In [61]:
df_AreaSummary = df_AASummaryTotals.groupby(['area_id'], as_index=False).agg({"acres":[np.sum],"job_spaces_orig":[np.sum],"job_spaces_new":[np.sum],"job_spaces_change":[np.sum],"resunits_orig":[np.sum],"resunits_new":[np.sum],"resunits_change":[np.sum]})
df_AreaSummary.columns = df_AreaSummary.columns.droplevel(1)

#read in csv with area names
df_AreaNames = pd.read_csv(AnalysisAreasTable)

#merge area names with area summary
df_AreaSummary_wNames = pd.DataFrame.merge(df_AreaNames, df_AreaSummary, left_on="AreaID", right_on="area_id")

#add total line to area summary with names
df_AreaSummary_wNames_wTotals = df_AreaSummary_wNames.append(df_AreaSummary_wNames.sum(numeric_only=True).rename('Total'))

#recalculate densities, round to nearest whole number (calculate before rounding of jobs/units)
df_AreaSummary_wNames_wTotals['density_hhemp_orig'  ] = ((df_AreaSummary_wNames_wTotals['job_spaces_orig'  ] + df_AreaSummary_wNames_wTotals['resunits_orig'     ]) / df_AreaSummary_wNames_wTotals['acres']).round(0)
df_AreaSummary_wNames_wTotals['density_hhemp_new'   ] = ((df_AreaSummary_wNames_wTotals['job_spaces_new'   ] + df_AreaSummary_wNames_wTotals['resunits_new'      ]) / df_AreaSummary_wNames_wTotals['acres']).round(0)
df_AreaSummary_wNames_wTotals['density_hhemp_change'] =  (df_AreaSummary_wNames_wTotals['density_hhemp_new'] - df_AreaSummary_wNames_wTotals['density_hhemp_orig']).round(0)

#recalculate based on rounded values, rounded to nearest hundred
df_AreaSummary_wNames_wTotals['job_spaces_orig'     ] = (df_AreaSummary_wNames_wTotals['job_spaces_orig'   ]).round(-2)
df_AreaSummary_wNames_wTotals['job_spaces_new'      ] = (df_AreaSummary_wNames_wTotals['job_spaces_new'    ]).round(-2)
df_AreaSummary_wNames_wTotals['job_spaces_change'   ] =  df_AreaSummary_wNames_wTotals['job_spaces_new'    ] - df_AreaSummary_wNames_wTotals['job_spaces_orig'] 

df_AreaSummary_wNames_wTotals['resunits_orig'       ] = (df_AreaSummary_wNames_wTotals['resunits_orig'     ]).round(-2)
df_AreaSummary_wNames_wTotals['resunits_new'        ] = (df_AreaSummary_wNames_wTotals['resunits_new'      ]).round(-2)
df_AreaSummary_wNames_wTotals['resunits_change'     ] =  df_AreaSummary_wNames_wTotals['resunits_new'      ] - df_AreaSummary_wNames_wTotals['resunits_orig'] 

#change index to Area ID
df_AreaSummary_wNames_wTotals = df_AreaSummary_wNames_wTotals.set_index(['area_id'])

#display table with numeric styling
display(df_AreaSummary_wNames_wTotals.style.format({"Acres":"{:,.1f}","job_spaces_orig":"{:,.0f}","job_spaces_new":"{:,.0f}","job_spaces_change":"{:,.0f}","resunits_orig":"{:,.0f}","resunits_new":"{:,.0f}","resunits_change":"{:,.0f}","density_hhemp_orig":"{:,.0f}","density_hhemp_new":"{:,.0f}","density_hhemp_change":"{:,.0f}"}))

#export to csv
df_AreaSummary_wNames_wTotals.to_csv(os.path.join(dir_results, r'SummaryTable_Area_wTotals.csv'))

,AreaID,AnalysisAreaName,acres,job_spaces_orig,job_spaces_new,job_spaces_change,resunits_orig,resunits_new,resunits_change,density_hhemp_orig,density_hhemp_new,density_hhemp_change
area_id,,,,,,,,,,,,
0.0,0.000000,American Fork - AMERICAN FORK CRT NC,43.947828,0,600,600,0,700,700,0,27,27
1.0,1.000000,American Fork - Meadows,321.655815,"4,500","6,600","2,100",200,"2,000","1,800",15,27,12
2.0,2.000000,American Fork - Timpanogos,117.049989,"2,300","3,500","1,200",0,"1,100","1,100",20,40,20
5.0,5.000000,Bluffdale - Independence,773.908744,"1,600","8,300","6,700",800,"7,100","6,300",3,20,17
6.0,6.000000,Bluffdale - Prison Site,152.349564,"2,000","6,900","4,900",0,"2,600","2,600",13,62,49
7.0,7.000000,Bountiful - 1100 North Center,24.793289,400,500,100,0,200,200,16,32,16
8.0,8.000000,Bountiful - Bountiful 200 West,32.231658,500,600,100,100,300,200,20,28,8
9.0,9.000000,Bountiful - Bountiful Downtown,70.605681,"2,600","2,900",300,200,500,300,39,48,9
12.0,12.000000,Centerville - Parrish Lane Center,102.774835,"2,600","2,800",200,0,500,500,25,32,7


# Class Summary

In [62]:
#Class table is aggregate of df_AASummaryTotals
df_ClassSummaryTable = df_AASummaryTotals.groupby(['ClassOrder','ClassDescription'], as_index=False).agg({"acres":[np.sum],"acres_redevelopable":[np.sum],"acres_developable":[np.sum],"job_spaces_orig":[np.sum],"job_spaces_new":[np.sum],"job_spaces_change":[np.sum],"resunits_orig":[np.sum],"resunits_new":[np.sum],"resunits_change":[np.sum], "job_spaces_undevelopable":[np.sum], "job_spaces_remain":[np.sum], "job_spaces_redeveloped_orig":[np.sum], "job_spaces_redeveloped_new":[np.sum], "job_spaces_developed":[np.sum], "resunits_undevelopable":[np.sum], "resunits_remain":[np.sum], "resunits_redeveloped_orig":[np.sum], "resunits_redeveloped_new":[np.sum], "resunits_developed":[np.sum]})

#drop aggregate description from column name (sum, etc)
df_ClassSummaryTable.columns = df_ClassSummaryTable.columns.droplevel(1)

#add totals row
df_ClassSummaryTable_wTotals = df_ClassSummaryTable.append(df_ClassSummaryTable.sum(numeric_only=True).rename('Total'))

#recalculate densities - calculate densities before rounding
df_ClassSummaryTable_wTotals['density_hhemp_orig'  ] = ((df_ClassSummaryTable_wTotals['job_spaces_orig'  ] + df_ClassSummaryTable_wTotals['resunits_orig'     ]) / df_ClassSummaryTable_wTotals['acres']).round(0)
df_ClassSummaryTable_wTotals['density_hhemp_new'   ] = ((df_ClassSummaryTable_wTotals['job_spaces_new'   ] + df_ClassSummaryTable_wTotals['resunits_new'      ]) / df_ClassSummaryTable_wTotals['acres']).round(0)
df_ClassSummaryTable_wTotals['density_hhemp_change'] =  (df_ClassSummaryTable_wTotals['density_hhemp_new'] - df_ClassSummaryTable_wTotals['density_hhemp_orig']).round(0)

#calculate rounded values
df_ClassSummaryTable_wTotals['job_spaces_orig'     ] =  (df_ClassSummaryTable_wTotals['job_spaces_orig'  ]).round(-2)
df_ClassSummaryTable_wTotals['job_spaces_new'      ] =  (df_ClassSummaryTable_wTotals['job_spaces_new'   ]).round(-2)
df_ClassSummaryTable_wTotals['job_spaces_change'   ] =   df_ClassSummaryTable_wTotals['job_spaces_new'   ] - df_ClassSummaryTable_wTotals['job_spaces_orig'] 

df_ClassSummaryTable_wTotals['resunits_orig'       ] =  (df_ClassSummaryTable_wTotals['resunits_orig'    ]).round(-2)
df_ClassSummaryTable_wTotals['resunits_new'        ] =  (df_ClassSummaryTable_wTotals['resunits_new'     ]).round(-2)
df_ClassSummaryTable_wTotals['resunits_change'     ] =   df_ClassSummaryTable_wTotals['resunits_new'     ] - df_ClassSummaryTable_wTotals['resunits_orig'] 

#fix last row name to be (HARD CODED CLASS ORDER-FIX!!)
df_ClassSummaryTable_wTotals.loc[(df_ClassSummaryTable_wTotals['ClassOrder'] == 28), 'ClassDescription'] = 'Total'
df_ClassSummaryTable_wTotals.loc[(df_ClassSummaryTable_wTotals['ClassOrder'] == 28), 'ClassOrder'] = 8

#set index to class order
df_ClassSummaryTable_wTotals = df_ClassSummaryTable_wTotals.set_index(['ClassOrder'])

df_ClassSummaryTable_wTotals['job_spaces_redev_add'       ] = df_ClassSummaryTable_wTotals['job_spaces_redeveloped_new'] -  df_ClassSummaryTable_wTotals['job_spaces_redeveloped_orig']
df_ClassSummaryTable_wTotals['resunits_redev_add'         ] = df_ClassSummaryTable_wTotals['resunits_redeveloped_new'  ] -  df_ClassSummaryTable_wTotals['resunits_redeveloped_orig'  ]

df_ClassSummaryTable_wTotals['job_spaces_redev_percentadd'] = df_ClassSummaryTable_wTotals['job_spaces_redev_add'      ] / (df_ClassSummaryTable_wTotals['job_spaces_redev_add'       ] + df_ClassSummaryTable_wTotals['job_spaces_developed'])
df_ClassSummaryTable_wTotals['job_spaces_dev_percentadd'  ] = df_ClassSummaryTable_wTotals['job_spaces_developed'      ] / (df_ClassSummaryTable_wTotals['job_spaces_redev_add'       ] + df_ClassSummaryTable_wTotals['job_spaces_developed'])

df_ClassSummaryTable_wTotals['resunits_redev_percentadd'  ] = df_ClassSummaryTable_wTotals['resunits_redev_add'        ] / (df_ClassSummaryTable_wTotals['resunits_redev_add'         ] + df_ClassSummaryTable_wTotals['resunits_developed'  ])
df_ClassSummaryTable_wTotals['resunits_dev_percentadd'    ] = df_ClassSummaryTable_wTotals['resunits_developed'        ] / (df_ClassSummaryTable_wTotals['resunits_redev_add'         ] + df_ClassSummaryTable_wTotals['resunits_developed'  ])

df_ClassSummaryTable_wTotals['Acres_Redev_Percent'        ] =  df_ClassSummaryTable_wTotals['acres_redevelopable'      ]                                                                / df_ClassSummaryTable_wTotals['acres']
df_ClassSummaryTable_wTotals['Acres_Dev_Percent'          ] =  df_ClassSummaryTable_wTotals['acres_developable'        ]                                                                / df_ClassSummaryTable_wTotals['acres']

df_ClassSummaryTable_wTotals['Acres_Unchanged_Percent'    ] = 1 - df_ClassSummaryTable_wTotals['Acres_Dev_Percent'] - df_ClassSummaryTable_wTotals['Acres_Redev_Percent']

df_ClassSummaryTable_wTotals = df_ClassSummaryTable_wTotals[['ClassDescription','acres','job_spaces_orig','job_spaces_new','job_spaces_change','resunits_orig','resunits_new','resunits_change','density_hhemp_orig','density_hhemp_new','density_hhemp_change','Acres_Unchanged_Percent','Acres_Redev_Percent','Acres_Dev_Percent','job_spaces_redev_percentadd','job_spaces_dev_percentadd','resunits_redev_percentadd','resunits_dev_percentadd']]

#display with formatted numbers
display(df_ClassSummaryTable_wTotals.style.format({"Acres":"{:,.0f}","job_spaces_orig":"{:,.0f}","job_spaces_new":"{:,.0f}","job_spaces_change":"{:,.0f}","resunits_orig":"{:,.0f}","resunits_new":"{:,.0f}","resunits_change":"{:,.0f}","density_hhemp_orig":"{:,.0f}","density_hhemp_new":"{:,.0f}","density_hhemp_change":"{:,.0f}","Acres_Unchanged_Percent":"{:,.0%}","Acres_Redev_Percent":"{:,.0%}","Acres_Dev_Percent":"{:,.0%}","job_spaces_redev_percentadd":"{:,.0%}","job_spaces_dev_percentadd":"{:,.0%}","resunits_redev_percentadd":"{:,.0%}","resunits_dev_percentadd":"{:,.0%}"}))

#export to csv
df_ClassSummaryTable_wTotals.to_csv(os.path.join(dir_results, r'SummaryTable_Class_wTotals.csv'))

,ClassDescription,acres,job_spaces_orig,job_spaces_new,job_spaces_change,resunits_orig,resunits_new,resunits_change,density_hhemp_orig,density_hhemp_new,density_hhemp_change,Acres_Unchanged_Percent,Acres_Redev_Percent,Acres_Dev_Percent,job_spaces_redev_percentadd,job_spaces_dev_percentadd,resunits_redev_percentadd,resunits_dev_percentadd
ClassOrder,,,,,,,,,,,,,,,,,,
1.0,CRT|Metropolitan Center,222.885898,"6,500","48,600","42,100","1,000","15,300","14,300",33,286,253,52%,26%,22%,53%,47%,54%,46%
2.0,LRT|Metropolitan Center,376.450392,"55,400","90,300","34,900","3,100","14,900","11,800",155,280,125,76%,6%,18%,23%,77%,25%,75%
3.0,NONTOD|Metropolitan Center,335.023971,"11,800","30,500","18,700","2,300","10,400","8,100",42,122,80,67%,22%,12%,62%,38%,65%,35%
4.0,CRT|Urban Center,875.384660,"13,100","45,300","32,200","1,500","17,800","16,300",17,72,55,68%,6%,26%,18%,82%,19%,81%
5.0,LRT|Urban Center,455.595806,"10,800","22,400","11,600","2,600","9,000","6,400",30,69,39,76%,8%,15%,30%,70%,35%,65%
6.0,NONTOD|Urban Center,9740.836088,"169,400","341,600","172,200","28,100","117,600","89,500",20,47,27,74%,5%,21%,17%,83%,18%,82%
7.0,CRT|City Center,345.427289,"1,400","12,600","11,200",200,"8,800","8,600",5,62,57,35%,4%,61%,4%,96%,6%,94%
8.0,LRT|City Center,1034.777562,"12,400","35,100","22,700","2,000","20,700","18,700",14,54,40,53%,15%,32%,26%,74%,32%,68%
9.0,NONTOD|City Center,18114.692859,"163,200","318,500","155,300","33,300","184,700","151,400",11,28,17,59%,5%,35%,8%,92%,12%,88%


In [63]:
df_AASummaryTotals
#df_ClassSummaryTable_wTotals

,county_id,area_id,class_id,ClassDescription,ClassOrder,parcel_id,acres,acres_redevelopable,acres_developable,job_spaces_orig,job_spaces_new,job_spaces_change,resunits_orig,resunits_new,resunits_change,job_spaces_undevelopable,job_spaces_remain,job_spaces_redeveloped_orig,job_spaces_redeveloped_new,job_spaces_developed,resunits_undevelopable,resunits_remain,resunits_redeveloped_orig,resunits_redeveloped_new,resunits_developed,density_hhemp_orig,density_hhemp_new,density_hhemp_change
0,1.0,7,8,NONTOD|City Center,9,57.0,24.793289,9.856027,0.800238,361,536,175,29,249,220,0.000000,299.695379,62.056418,220.0,17.0,27.659109,1.437227,0.000000,204.0,16.0,15.7,31.7,16.0
1,1.0,8,8,NONTOD|City Center,9,136.0,32.231658,5.720723,2.554351,517,598,81,124,290,166,25.948782,388.703519,103.266821,127.0,57.0,0.000000,120.824606,3.639046,118.0,52.0,19.9,27.6,7.7
2,1.0,9,8,NONTOD|City Center,9,282.0,70.605681,6.749477,10.670130,2622,2907,285,156,503,347,196.972892,2322.799815,102.985383,150.0,238.0,0.000000,143.980267,12.998219,139.0,221.0,39.3,48.3,9.0
3,1.0,12,8,NONTOD|City Center,9,79.0,102.774835,14.250215,10.548388,2588,2786,198,9,521,512,53.819069,2179.193155,355.944689,318.0,235.0,0.000000,8.087687,0.999848,295.0,218.0,25.3,32.2,6.9
4,1.0,13,2,CRT|NA,10,125.0,123.802694,22.230292,80.865414,302,1864,1562,128,2293,2165,0.000000,4.969098,298.028888,401.0,1459.0,0.000000,124.610140,4.254629,467.0,1702.0,3.5,33.6,30.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,4.0,117,2,CRT|NA,10,448.0,46.845719,1.905297,8.570689,301,489,188,239,454,215,0.778908,301.133570,0.000000,34.0,154.0,5.978531,228.782916,4.791317,40.0,180.0,11.5,20.1,8.6
254,4.0,161,8,NONTOD|City Center,9,765.0,511.584632,0.000000,409.820761,305,9456,9151,161,8658,8497,10.147910,295.053134,0.000000,0.0,9151.0,39.203800,122.788308,0.000000,0.0,8497.0,0.9,35.4,34.5
255,4.0,179,8,NONTOD|City Center,9,228.0,115.461824,0.138183,44.639835,2711,3710,999,5,931,926,0.000000,2711.486008,0.000000,3.0,996.0,0.000000,4.469958,0.543201,2.0,925.0,23.5,40.2,16.7
256,4.0,180,8,NONTOD|City Center,9,1584.0,331.285589,21.125226,139.462872,2554,6139,3585,636,3908,3272,25.153141,2529.806727,0.000000,471.0,3114.0,48.523885,530.632614,57.379042,438.0,2891.0,9.6,30.3,20.7


# Summary by County

In [64]:
df_CountySummary = df_AASummaryTotals.groupby(['county_id'], as_index=False).agg({"acres":[np.sum],"job_spaces_orig":[np.sum],"job_spaces_new":[np.sum],"job_spaces_change":[np.sum],"resunits_orig":[np.sum],"resunits_new":[np.sum],"resunits_change":[np.sum]})
df_CountySummary.columns = df_CountySummary.columns.droplevel(1)

#read in csv with area names
df_CountyNames = pd.read_csv(CountiesTable)

#merge area names with area summary
df_CountySummary_wNames = pd.DataFrame.merge(df_CountyNames, df_CountySummary, on="county_id")

#add total line to area summary with names
df_CountySummary_wNames_wTotals = df_CountySummary_wNames.append(df_CountySummary_wNames.sum(numeric_only=True).rename('Total'))

#recalculate densities, round to nearest whole number (calculate before rounding of jobs/units)
df_CountySummary_wNames_wTotals['density_hhemp_orig'  ] = ((df_CountySummary_wNames_wTotals['job_spaces_orig'  ] + df_CountySummary_wNames_wTotals['resunits_orig'     ]) / df_CountySummary_wNames_wTotals['acres']).round(0)
df_CountySummary_wNames_wTotals['density_hhemp_new'   ] = ((df_CountySummary_wNames_wTotals['job_spaces_new'   ] + df_CountySummary_wNames_wTotals['resunits_new'      ]) / df_CountySummary_wNames_wTotals['acres']).round(0)
df_CountySummary_wNames_wTotals['density_hhemp_change'] =  (df_CountySummary_wNames_wTotals['density_hhemp_new'] - df_CountySummary_wNames_wTotals['density_hhemp_orig']).round(0)

#recalculate based on rounded values, rounded to nearest hundred
df_CountySummary_wNames_wTotals['job_spaces_orig'     ] = (df_CountySummary_wNames_wTotals['job_spaces_orig'   ]).round(-2)
df_CountySummary_wNames_wTotals['job_spaces_new'      ] = (df_CountySummary_wNames_wTotals['job_spaces_new'    ]).round(-2)
df_CountySummary_wNames_wTotals['job_spaces_change'   ] =  df_CountySummary_wNames_wTotals['job_spaces_new'    ] - df_CountySummary_wNames_wTotals['job_spaces_orig'] 

df_CountySummary_wNames_wTotals['resunits_orig'       ] = (df_CountySummary_wNames_wTotals['resunits_orig'     ]).round(-2)
df_CountySummary_wNames_wTotals['resunits_new'        ] = (df_CountySummary_wNames_wTotals['resunits_new'      ]).round(-2)
df_CountySummary_wNames_wTotals['resunits_change'     ] =  df_CountySummary_wNames_wTotals['resunits_new'      ] - df_CountySummary_wNames_wTotals['resunits_orig'] 

#change index to Area ID
df_CountySummary_wNames_wTotals = df_CountySummary_wNames_wTotals.set_index(['CO_ORDER'])

#display table with numeric styling
#display(df_CountySummary_wNames_wTotals.style.format({"acres":"{:,.1f}","job_spaces_orig":"{:,.0f}","job_spaces_new":"{:,.0f}","job_spaces_change":"{:,.0f}","resunits_orig":"{:,.0f}","resunits_new":"{:,.0f}","resunits_change":"{:,.0f}","density_hhemp_orig":"{:,.0f}","density_hhemp_new":"{:,.0f}","density_hhemp_change":"{:,.0f}"}))

#export to csv
df_CountySummary_wNames_wTotals.to_csv(os.path.join(dir_results, r'SummaryTable_County_wTotals.csv'))

In [65]:
df_CountySummary_wNames_wTotals

,county_id,CO_NAME,acres,job_spaces_orig,job_spaces_new,job_spaces_change,resunits_orig,resunits_new,resunits_change,density_hhemp_orig,density_hhemp_new,density_hhemp_change
CO_ORDER,,,,,,,,,,,,
1.0,3.0,Weber,5509.421159,48500.0,100900.0,52400.0,10000.0,45500.0,35500.0,11.0,27.0,16.0
2.0,1.0,Davis,5605.724290,41600.0,128900.0,87300.0,10200.0,68300.0,58100.0,9.0,35.0,26.0
3.0,2.0,Salt Lake,17181.376308,302500.0,584900.0,282400.0,44300.0,230200.0,185900.0,20.0,47.0,27.0
4.0,4.0,Utah,6669.248690,89100.0,185200.0,96100.0,18600.0,86700.0,68100.0,16.0,41.0,25.0
10.0,10.0,NaN,34965.770447,481600.0,999900.0,518300.0,83100.0,430700.0,347600.0,16.0,41.0,25.0


# Display HH/Emp Growth by Analysis Areas

In [66]:
df_AnalysisAreas = pd.read_csv(AnalysisAreasTable)

sdf_AnalysisAreas = pd.DataFrame.merge(sdf_AAid, df_AnalysisAreas, left_on='area_id', right_on='AreaID')

sdf_AANewHHEmp = pd.DataFrame.merge(sdf_AnalysisAreas, df_AASummaryTotals, on=['area_id','class_id'])
sdf_AANewHHEmp

,FID,Shape_Leng,Shape_Area,area_id,class_id,SHAPE,AreaID,AnalysisAreaName,county_id,ClassDescription,ClassOrder,parcel_id,acres,acres_redevelopable,acres_developable,job_spaces_orig,job_spaces_new,job_spaces_change,resunits_orig,resunits_new,resunits_change,job_spaces_undevelopable,job_spaces_remain,job_spaces_redeveloped_orig,job_spaces_redeveloped_new,job_spaces_developed,resunits_undevelopable,resunits_remain,resunits_redeveloped_orig,resunits_redeveloped_new,resunits_developed,density_hhemp_orig,density_hhemp_new,density_hhemp_change
0,2,0.081911,3.457529e-04,5,8,"{'rings': [[[-111.90309291399996, 40.486177586...",5,Bluffdale - Independence,2.0,NONTOD|City Center,9,328.0,773.908744,16.819438,285.853497,1564,8319,6755,792,7066,6274,51.285191,1509.788278,3.000221,375.0,6383.0,698.211053,92.998899,0.930432,348.0,5927.0,3.0,19.9,16.9
1,3,0.032693,6.718693e-07,43,8,"{'rings': [[[-112.04360043899999, 40.516117516...",43,Herriman - Herriman Olympia Hills,2.0,NONTOD|City Center,9,24.0,1.560506,0.000000,0.517910,0,11,11,0,10,10,0.000000,0.000000,0.000000,0.0,11.0,0.051602,0.093409,0.000000,0.0,10.0,0.0,13.5,13.5
2,4,0.030717,5.078766e-05,44,8,"{'rings': [[[-111.94214447999997, 40.460508711...",44,Herriman - Herriman Porter Rockwell Center,2.0,NONTOD|City Center,9,87.0,117.602071,0.000000,101.241818,0,2260,2260,0,2099,2099,0.000000,0.000000,0.000000,0.0,2260.0,0.000000,0.000000,0.000000,0.0,2099.0,0.0,37.1,37.1
3,5,0.052583,1.209422e-04,45,8,"{'rings': [[[-112.01070827299998, 40.521918312...",45,Herriman - Herriman Town,2.0,NONTOD|City Center,9,187.0,263.963647,0.000000,234.773939,103,5345,5242,94,4962,4868,0.000000,103.769387,0.000000,0.0,5242.0,0.000000,94.679241,0.000000,0.0,4868.0,0.7,39.0,38.3
4,6,0.014034,9.153544e-06,190,8,"{'rings': [[[-111.98691036499997, 40.650388205...",190,Taylorsville - Kearns City Center,2.0,NONTOD|City Center,9,47.0,16.974138,1.461712,4.686203,197,306,109,40,167,127,0.000000,170.509271,26.815557,32.0,104.0,0.000000,40.918667,0.000000,30.0,97.0,14.0,27.9,13.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,289,0.160173,2.067631e-04,147,9,"{'rings': [[[-111.90748598099998, 40.768529726...",147,Salt Lake City - Salt Lake City CBD,2.0,NONTOD|Metropolitan Center,3,1300.0,335.023971,72.272966,38.888309,11826,30491,18665,2329,10449,8120,281.031534,10195.452119,1349.525084,13013.0,7002.0,3.252943,2259.585471,66.383806,5323.0,2864.0,42.3,122.2,79.9
281,290,0.031583,7.897752e-05,147,1,"{'rings': [[[-111.90269131299999, 40.762673174...",147,Salt Lake City - Salt Lake City CBD,2.0,CRT|Metropolitan Center,1,1143.0,222.885898,57.965787,50.011241,6452,48566,42114,991,15282,14291,679.217735,4918.518600,855.196837,23067.0,19902.0,35.848240,924.087573,31.187145,7689.0,6634.0,33.4,286.5,253.1
282,291,0.030149,4.829866e-05,147,1,"{'rings': [[[-111.89878362799999, 40.772698415...",147,Salt Lake City - Salt Lake City CBD,2.0,CRT|Metropolitan Center,1,1143.0,222.885898,57.965787,50.011241,6452,48566,42114,991,15282,14291,679.217735,4918.518600,855.196837,23067.0,19902.0,35.848240,924.087573,31.187145,7689.0,6634.0,33.4,286.5,253.1
283,292,0.085443,2.256532e-04,147,5,"{'rings': [[[-111.88250224899997, 40.763937603...",147,Salt Lake City - Salt Lake City CBD,2.0,LRT|Metropolitan Center,2,2366.0,376.450392,22.190871,67.231252,55417,90345,34928,3063,14919,11856,12764.725767,41994.827102,658.270004,8831.0,26755.0,35.641804,3022.784538,5.531244,2943.0,8918.0,155.3,279.6,124.3


In [67]:
#create map centered on Salt Lake
map_parcels_areas_add = gis.map('Salt Lake')

#expression to classify employment growth
arcade_expression_jobspaceschange = ("var v = $feature.job_spaces_change;"
                                     "if      (v<0    ) { return 'class0'; }"
                                     "if      (v<2500 ) { return 'class1'; }"
                                     "else if (v<5000 ) { return 'class2'; }"
                                     "else if (v<10000) { return 'class3'; }"
                                     "else              { return 'class4'; }")

#symbology for enrollment classes
uv_jobspaceschange = [{"value":"class0", "label":"Job Decline"     , "symbol":{"type":"esriSFS","color":[  0,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                      {"value":"class1", "label":"0 to 2,500"      , "symbol":{"type":"esriSFS","color":[130,165,217,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                      {"value":"class2", "label":"2,500 to 5,000"  , "symbol":{"type":"esriSFS","color":[140,125,164,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                      {"value":"class3", "label":"5,000 to 10,000" , "symbol":{"type":"esriSFS","color":[149,85,111,168],  "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                      {"value":"class4", "label":"More than 10,000", "symbol":{"type":"esriSFS","color":[168,4,4,168],     "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]


#define sdf layer
sdf_AANewHHEmp.spatial.plot(map_widget = map_parcels_areas_add,
                            renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                            unique_values=uv_jobspaceschange,
                            arcade_expression=arcade_expression_jobspaceschange,
                            default_symbol="" #don't include an 'other' category
                            )

#define map characteristics
map_parcels_areas_add.layout.height='500px'
map_parcels_areas_add.legend=True

#map title
display(Markdown('<h2><center>Job Growth by Analysis Area</center></h2>'))

#display map
map_parcels_areas_add

<h2><center>Job Growth by Analysis Area</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

In [68]:
#create map centered on Salt Lake
map_parcels_areas_addHH = gis.map('Salt Lake')

#expression to classify employment growth
arcade_expression_resunitschange = ("var v = $feature.resunits_change;"
                                    "if      (v<0   ) { return 'class0'; }"
                                    "if      (v<1000) { return 'class1'; }"
                                    "else if (v<2000) { return 'class2'; }"
                                    "else if (v<5000) { return 'class3'; }"
                                    "else             { return 'class4'; }")

#symbology for enrollment classes
uv_resunitschange = [{"value":"class0", "label":"HH Decline"     , "symbol":{"type":"esriSFS","color":[  0,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                     {"value":"class1", "label":"0 to 1,000"     , "symbol":{"type":"esriSFS","color":[130,165,217,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                     {"value":"class2", "label":"1,000 to 2,000" , "symbol":{"type":"esriSFS","color":[140,125,164,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                     {"value":"class3", "label":"2,000 to 5,000" , "symbol":{"type":"esriSFS","color":[149, 85,111,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                     {"value":"class4", "label":"More than 5,000", "symbol":{"type":"esriSFS","color":[168,  4,  4,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]


#define sdf layer
sdf_AANewHHEmp.spatial.plot(map_widget = map_parcels_areas_addHH,
                            renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                            unique_values=uv_resunitschange,
                            arcade_expression=arcade_expression_resunitschange,
                            default_symbol="" #don't include an 'other' category
                            )

#define map characteristics
map_parcels_areas_addHH.layout.height='500px'
map_parcels_areas_addHH.legend=True

#map title
display(Markdown('<h2><center>Household Growth by Analysis Area</center></h2>'))

#display map
map_parcels_areas_addHH

<h2><center>Household Growth by Analysis Area</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

In [69]:
sdf_AA

,FID,FID_Munici,COUNTYNBR,NAME,COUNTYSEAT,SHORTDESC,UPDATED,FIPS,ENTITYNBR,SALESTAXID,IMSCOLOR,MINNAME,POPLASTCEN,POPLASTEST,GNIS,GlobalID,SHAPE_Leng,FID_TODCen,AreaName,AreaType,Area,ClassName,Shape_Le_1,Shape_Area,AnalysisAr,Analysis_1,SHAPE
0,0,5,18,Bluffdale,0,BLUFFDALE (SL CO),2018-11-15,06810,3020.0,019,4,,7693,14699,1425844,{E52CE320-1532-4A57-8DB9-DFA819FBB85B},0.303290,64,Bluffdale - Downtown Riverton,City Center,WFRC MPO,NONTOD|City Center,0.002517,1.025230e-07,Bluffdale - Downtown Riverton,NONTOD|City Center,"{""rings"": [[[-111.93828708318233, 40.500645841..."
1,1,5,18,Bluffdale,0,BLUFFDALE (SL CO),2018-11-15,06810,3020.0,019,4,,7693,14699,1425844,{E52CE320-1532-4A57-8DB9-DFA819FBB85B},0.303290,77,Bluffdale - Herriman Porter Rockwell Center,City Center,WFRC MPO,NONTOD|City Center,0.014422,1.756575e-06,Bluffdale - Herriman Porter Rockwell Center,NONTOD|City Center,"{""rings"": [[[-111.94214447958404, 40.460508711..."
2,2,5,18,Bluffdale,0,BLUFFDALE (SL CO),2018-11-15,06810,3020.0,019,4,,7693,14699,1425844,{E52CE320-1532-4A57-8DB9-DFA819FBB85B},0.303290,82,Bluffdale - Independence,City Center,WFRC MPO,NONTOD|City Center,0.081911,3.457529e-04,Bluffdale - Independence,NONTOD|City Center,"{""rings"": [[[-111.90309291350712, 40.486177585..."
3,3,5,18,Bluffdale,0,BLUFFDALE (SL CO),2018-11-15,06810,3020.0,019,4,,7693,14699,1425844,{E52CE320-1532-4A57-8DB9-DFA819FBB85B},0.303290,121,Bluffdale - Prison Site,Urban Center,WFRC MPO,NONTOD|Urban Center,0.060180,6.782125e-05,Bluffdale - Prison Site,NONTOD|Urban Center,"{""rings"": [[[-111.90309291350712, 40.486177585..."
4,4,7,18,Herriman,0,HERRIMAN TOWN,2019-11-22,34970,3035.0,060,1,,22018,44877,1428675,{3E78F48D-80B9-44DA-AC9F-836A7B4F1B8E},0.617524,76,Herriman - Herriman Olympia Hills,City Center,WFRC MPO,NONTOD|City Center,0.032693,6.718727e-07,Herriman - Herriman Olympia Hills,NONTOD|City Center,"{""rings"": [[[-112.05317660771425, 40.526060430..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,289,169,18,West Valley City,0,WEST VALLEY CITY,2012-02-02,83470,3120.0,167,3,West Valley,133796,136401,1437843,{D013E6C0-091A-4BB6-BBA7-561D1052928C},0.614523,191,West Valley City - Fairbourne Station,Urban Center,WFRC MPO,LRT|Urban Center,0.028093,4.651836e-05,West Valley City - Fairbourne Station,LRT|Urban Center,"{""rings"": [[[-111.95988335513272, 40.697973328..."
290,290,223,18,Salt Lake City,1,SALT LAKE CITY,2018-11-14,67000,3070.0,122,4,Salt Lake City,190749,200591,1454997,{B57017A5-AD7A-4135-BA73-C3A6DDAFE2B1},1.346029,46,Salt Lake City - 1940 W NORTH TEMPLE LRT NC,,WFRC MPO,LRT|NA,0.020945,2.054510e-05,Salt Lake City - 1940 W NORTH TEMPLE LRT NC,LRT|NA,"{""rings"": [[[-111.94851766915508, 40.773298628..."
291,291,223,18,Salt Lake City,1,SALT LAKE CITY,2018-11-14,67000,3070.0,122,4,Salt Lake City,190749,200591,1454997,{B57017A5-AD7A-4135-BA73-C3A6DDAFE2B1},1.346029,205,Salt Lake City - North Temple Center,City Center,WFRC MPO,LRT|City Center,0.020945,2.054510e-05,Salt Lake City - North Temple Center,LRT|City Center,"{""rings"": [[[-111.94851766915508, 40.773298628..."
292,292,226,29,Roy,0,ROY CITY,2018-11-09,65110,3090.0,037,4,,37669,38773,1432035,{31203C12-2594-48F6-9A14-44C48F31B40E},0.263489,208,Roy - Midland Drive - 3500 West,City Center,WFRC MPO,NONTOD|City Center,0.002902,2.813621e-11,Roy - Midland Drive - 3500 West,NONTOD|City Center,"{""rings"": [[[-112.06303223395042, 41.164103772..."


In [71]:
df_AASummary.to_csv(os.path.join(dir_results,"dfAASummary.csv"),index=False)
df_AADevCat .to_csv(os.path.join(dir_results,"dfAADevCat.csv" ),index=False)